In [1]:
import sys
sys.path.insert(0, '/Users/mondormo/stackla17/stackla-web/etc/python')


from multilabels_model import keras_model_fn, train_input_fn, eval_input_fn, _load_data
import os, re, csv, math, codecs
import numpy as np
import pandas as pd

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout, GlobalMaxPooling1D, Conv1D, MaxPooling1D, BatchNormalization
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [2]:
max_features = 50000
input_dim = 150
n_classes = 2
num_epochs = 10
batch_size = 256
embed_dim = 300

hyperparameters = {
    "input_dim": input_dim,
    "n_classes": n_classes, 
    "num_epochs": num_epochs,
    "batch_size": batch_size,
    "pretrain_embedding": "s3://sagemaker-us-east-1-953585160895/pretrain-embedding/wiki.simple.vec",
    "embed_dim": embed_dim
}


DATA_PATH='data'
model = keras_model_fn(hyperparameters)

file_path="prototype-cnn-weights.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)
callbacks_list = [checkpoint, early]





downloading s3://sagemaker-us-east-1-953585160895/pretrain-embedding/wiki.simple.vec to wiki.simple.vec
data/wiki.simple.vec already exists
Building embedding_index...
embedding_index length: 111051
Building word_index...
word_index length: 6175
Building embedding_matrix...
embedding_matrix shape (6176, 300)


In [3]:
train_x, train_y = _load_data('data/training', 'train', n_classes)
val_x, val_y = _load_data('data/training', 'eval', n_classes)

print('train_x: %s, train_y: %s, val_x: %s, val_y: %s' % (train_x.shape,train_y.shape, val_x.shape, val_y.shape))

train_x: (264, 150), train_y: (264, 2), val_x: (66, 150), val_y: (66, 2)


In [4]:
model.fit(train_x, train_y, batch_size=batch_size, epochs=num_epochs, validation_split=0.2, callbacks=callbacks_list)


/Users/mondormo/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 211 samples, validate on 53 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 0.70395, saving model to prototype-cnn-weights.best.hdf5
211/211 [==============================] - 5s 23ms/step - loss: 0.7423 - acc: 0.5071 - f1: 0.5720 - val_loss: 0.7040 - val_acc: 0.5377 - val_f1: 0.1695
Epoch 2/10

Epoch 00002: val_loss improved from 0.70395 to 0.66173, saving model to prototype-cnn-weights.best.hdf5
211/211 [==============================] - 2s 9ms/step - loss: 0.7585 - acc: 0.5213 - f1: 0.3176 - val_loss: 0.6617 - val_acc: 0.5943 - val_f1: 0.4557
Epoch 3/10

Epoch 00003: val_loss improved from 0.66173 to 0.64743, saving model to prototype-cnn-weights.best.hdf5
211/211 [==============================] - 2s 9ms/step - loss: 0.6940 - acc: 0.5521 - f1: 0.4553 - val_loss: 0.6474 - val_acc: 0.6509 - val_f1: 0.6667
Epoch 4/10

Epoch 00004: val_loss did not improve
211/211 [==============================] - 2s 10ms/step - loss: 0.6625 - acc: 0.5900 - f1: 0.6095 - val_los

In [5]:
model.load_weights(file_path)
model.evaluate(val_x, val_y, verbose=1)

66/66 [==============================] - 0s 2ms/step


[0.4855870243274804, 0.8257575757575758, 0.8025366176258434]